In [12]:
import random
import pandas as pd

In [16]:
def get_item_z():
    item = {}
    for i in range(3):
        item[i] = random.uniform(35.5, 36.8)
    for i in range(3, 8):
        item[i] = random.uniform(35.5, 36.8)
    
    j = 8
    for i in range(3, 8):
        temp = random.uniform(item[i] - 0.25, item[i] + 0.25)
        item[j] = temp
        j += 1
    
    return item

zdorovie = []
for i in range(50):
    zdorovie.append(get_item_z())

z_df = pd.DataFrame(zdorovie)
z_df['target'] = 0
z_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,target
0,36.488302,35.996469,35.861259,35.995202,35.965815,36.167530,36.533561,36.107311,35.869104,35.948878,35.934109,36.600438,35.955749,0
1,35.671766,35.819769,35.784392,36.037948,36.761493,36.754503,36.332396,35.705054,35.991069,36.640264,36.832995,36.512174,35.729384,0
2,36.425982,35.532480,36.083828,36.518652,36.299263,36.192020,36.028364,36.069094,36.459077,36.246916,36.008925,36.231236,36.242812,0
3,36.610815,35.958194,36.427172,36.082351,35.703789,36.087908,36.405219,36.292415,36.164554,35.550190,36.219439,36.263623,36.344505,0
4,36.281928,36.406792,35.639945,35.597612,36.208097,35.549823,36.060923,36.265613,35.714857,36.280426,35.716046,36.126089,36.202576,0


In [18]:
def get_item_b():
    item = {}
    for i in range(3):
        item[i] = random.uniform(34.5, 37.8)
    for i in range(3, 8):
        item[i] = random.uniform(34.5, 37.8)
    
    j = 8
    for i in range(3, 8):
        temp = random.uniform(item[i] - 0.25, item[i] + 0.5)
        item[j] = temp
        j += 1
    
    return item

bolnie = []
for i in range(50):
    bolnie.append(get_item_b())

b_df = pd.DataFrame(bolnie)
b_df['target'] = 1
b_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,target
0,37.500186,36.570426,35.808293,37.750586,34.793027,36.293386,35.976883,37.085404,37.807165,34.621843,36.081818,36.225960,37.174591,1
1,35.866491,36.356645,37.181925,36.954099,34.672211,35.395097,37.654211,35.443189,36.861901,34.641711,35.477089,37.620644,35.689431,1
2,34.916042,35.517179,34.597992,34.649241,37.572113,37.251531,37.563734,37.539228,34.574355,37.703786,37.714992,37.631671,37.685327,1
3,34.917507,35.753972,34.573616,37.794173,34.853067,35.760221,34.645185,36.637190,37.589797,34.914626,35.547285,35.004040,36.675335,1
4,36.497986,35.417832,37.583210,35.140682,37.541957,35.106510,36.265813,35.372863,35.503628,37.318985,35.200133,36.286638,35.691450,1


In [26]:
res = pd.concat([z_df, b_df])
res.to_csv('data.csv', encoding='utf-8', index=False)

In [36]:
import sqlite3

def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d
 
conn = sqlite3.connect("vkr.db")
conn.row_factory = dict_factory
cursor = conn.cursor()

columns = ['t' + str(i) + ' real' for i in range(13)]
# Создание таблицы
cursor.execute('CREATE TABLE IF NOT EXISTS train_data (' + ', '.join(columns) + ', target integer)')

# cursor.executemany("INSERT INTO train_data VALUES (" + ','.join(['?' for i in range(14)]) + ")", res.values)
# conn.commit()

In [38]:
cursor.execute('select * from train_data')
pd.DataFrame(cursor.fetchall()).head()

,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,target
0,36.488302,35.996469,35.861259,35.995202,35.965815,36.167530,36.533561,36.107311,35.869104,35.948878,35.934109,36.600438,35.955749,0
1,35.671766,35.819769,35.784392,36.037948,36.761493,36.754503,36.332396,35.705054,35.991069,36.640264,36.832995,36.512174,35.729384,0
2,36.425982,35.532480,36.083828,36.518652,36.299263,36.192020,36.028364,36.069094,36.459077,36.246916,36.008925,36.231236,36.242812,0
3,36.610815,35.958194,36.427172,36.082351,35.703789,36.087908,36.405219,36.292415,36.164554,35.550190,36.219439,36.263623,36.344505,0
4,36.281928,36.406792,35.639945,35.597612,36.208097,35.549823,36.060923,36.265613,35.714857,36.280426,35.716046,36.126089,36.202576,0


In [46]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
model = SVC(gamma='scale')

cursor.execute('select * from train_data')
xy = pd.DataFrame(cursor.fetchall())

train_x = xy[['t' + str(i) for i in range(13)]].values
train_y = xy['target']

model.fit(train_x, train_y)

accuracy_score(train_y, model.predict(train_x))

0.98